In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 31.5 MB/s 
     |████████████████████████████████| 596 kB 38.5 MB/s 
     |████████████████████████████████| 895 kB 51.1 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import os 
import json
import requests
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'
os.mkdir('squad') #OMIT IF U ALRDY HAVE SQUAD FOLDER
for file in ['train-v2.0.json', 'dev-v2.0.json']:
  res = requests.get(f'{url}{file}')
  #after make request we save file to squad
  with open(f'squad/{file}', 'wb') as f: #wb for writing binary file
    for chunk in res.iter_content(chunk_size=4):
      f.write(chunk)
    res
with open('squad/train-v2.0.json', 'rb') as f:
  squad_dict = json.load(f)
def read_data(path):
  with open(path, 'rb') as f:
    squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
  for group in squad_dict['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        if 'plausible_answers' in qa.keys():
          access = 'plausible_answers'
        else:
          access = 'answers'
        for answer in qa[access]:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)
#We want to do this for both the training and validation set so lets insert this into a function.
    return contexts, questions, answers
train_contexts, train_questions, train_answers = read_data('squad/train-v2.0.json')
test_contexts, test_questions, test_answers = read_data('squad/dev-v2.0.json')
def add_end_index(answers, contexts):
  for answer,context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)
    #Now we add in the logic where the characters index are off 
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx 
    else:
      for n in [1,2]:
        if context[start_idx-n,end_idx-n] == gold_text:
          answer['answer_start'] = context[start_idx-n]
          answer['answer_end'] = context[end_idx - n]
add_end_index(train_answers, train_contexts)
add_end_index(test_answers, test_contexts)
import transformers
from transformers import DistilBertTokenizerFast 
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encoding = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
#This is is now going to merge the two strings together but there will be a seperator token 
#Truncation and padding set to true of course to make sure they are all of the same size 
test_encoding = tokenizer(test_contexts, test_questions, truncation=True, padding=True)
def add_token_positions(encodings, answer):
  start_positions = []
  end_positions = []
  for i in range(len(answer)):
    start_positions.append(encodings.char_to_token(i, answer[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answer[i]['answer_end']))
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length #Since there are a chances that context gets broken off to hit question in 
    go_back = -1
    while end_positions[-1] is None:
      end_positions[-1] = encodings.char_to_token(i, answer[i]['answer_end']-go_back)
      go_back +=1
  encodings.update({
      'start_positions':start_positions,
      'end_positions':end_positions
  })
add_token_positions(train_encoding, train_answers)
add_token_positions(test_encoding, test_answers)
train_encoding.keys()

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [7]:
import numpy as np
input_type_ids = np.zeros(shape=(753,451))
input_ids = train_encoding['input_ids']
for i in range(len(input_ids)):
  indice = input_ids[i].index(102)
  input_type_ids[i][indice:452] = 1

In [8]:
input_type_ids

array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [13]:
import tensorflow_hub as hub 
import tensorflow as tf
input_ids = train_encoding['input_ids']
input_maskk = train_encoding['attention_mask']
start_positions_list = train_encoding['start_positions']
end_positions_list = train_encoding['end_positions']
vals = np.array(input_ids)
vals.shape #from examination we see that although we have 753 instances, the inputs are of size 451 on the column dir which is what
#our inputs in tensorflow care about
input_word_ids = tf.keras.layers.Input(shape=(451,), dtype=tf.int32, name = 'input_word_ids')
input_mask = tf.keras.layers.Input(shape=(451,), dtype = tf.int32, name = 'input_mask')
input_type_idss = tf.keras.layers.Input(shape=(451,), dtype = tf.int32, name = 'input_type_ids')
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_idss])
#This model is expecting 3 inputs, this includes an 'input_type_ids' which I believe corresponds to segment

In [14]:
#Here x is really just the input word ids, input mask, and input type ids all in a list!
x = [np.array(input_ids), np.array(input_maskk), np.array(input_type_ids)]
y = [np.array(start_positions_list), np.array(end_positions_list)]
start_layer_between = tf.keras.layers.Dense(1024, activation = 'relu')(sequence_output)
end_layer_between = tf.keras.layers.Dense(1024, activation = 'relu')(sequence_output)
start_logits = tf.keras.layers.Dense(1, name='start_logit', use_bias=False)(start_layer_between)
start_logits = tf.keras.layers.Flatten()(start_logits)
end_logits = tf.keras.layers.Dense(1, name = 'end_logit',use_bias=False)(end_layer_between)
end_logits = tf.keras.layers.Flatten()(end_logits)
start_prob = tf.keras.layers.Activation(tf.keras.activations.softmax)(start_logits)
end_prob = tf.keras.layers.Activation(tf.keras.activations.softmax)(end_logits)
model = tf.keras.Model(inputs = [input_word_ids, input_mask, input_type_idss], outputs = [start_prob, end_prob])
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.98,epsilon=1e-9)
model.compile(optimizer = optimizer, loss = [loss, loss])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 451)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 451)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 451)]        0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 451, 768)]                'input_mask[0][0]',       

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
model.fit(x, y, epochs = 10)

Epoch 1/10
24/24 [==============================] - 61s 2s/step - loss: 11.6820 - activation_2_loss: 5.8721 - activation_3_loss: 5.8099
Epoch 2/10
24/24 [==============================] - 52s 2s/step - loss: 10.9596 - activation_2_loss: 5.5205 - activation_3_loss: 5.4390
Epoch 3/10
24/24 [==============================] - 52s 2s/step - loss: 10.4353 - activation_2_loss: 5.2611 - activation_3_loss: 5.1743
Epoch 4/10
24/24 [==============================] - 53s 2s/step - loss: 10.0466 - activation_2_loss: 5.0564 - activation_3_loss: 4.9902
Epoch 5/10
24/24 [==============================] - 53s 2s/step - loss: 9.7409 - activation_2_loss: 4.8849 - activation_3_loss: 4.8560
Epoch 6/10
24/24 [==============================] - 53s 2s/step - loss: 9.4865 - activation_2_loss: 4.7350 - activation_3_loss: 4.7515
Epoch 7/10
24/24 [==============================] - 53s 2s/step - loss: 9.2638 - activation_2_loss: 4.6017 - activation_3_loss: 4.6620
Epoch 8/10
24/24 [==============================] -

In [ ]:
#note that my gpu was not able to handle the above task of actually using the pretrained model for this. This is why we are struggling lol
#We also need a lot more training data. I am about to redo this process but use another dataset with a lot more instances to do better!
